In [4]:
import itertools
import re
import numpy as np
import pandas as pd
import openai
import pandas as pd
import itertools
import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client
from transformers import pipeline
from groq import Groq


In [5]:
def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    '''

    ingredients_combinations = []
    powerpowerset = []
    ingredients = re.split(r',', ingredients_input.strip())
    ingredients_list = list(set(ingredient.strip() for ingredient in ingredients))
    for r in range(len(ingredients_list)+1):
        combinations = itertools.combinations(ingredients_list, r)
        #powerset.extend(subset for subset in combinations if len(subset) > 1)
        for comb in combinations:
            if len(comb) > 1:
                if len(comb) < 3:
                    ingredients_combinations.append(comb)
                else:
                    powerpowerset.append(comb)
                    for power in powerpowerset:
                        lowerset = []
                        combins = itertools.combinations(power, 2)
                        for arrange in combins:
                            lowerset.append(arrange)
                    ingredients_combinations.append(lowerset)
    return ingredients_combinations

combinations = combinations_of_two('chicken, onion, garlic, cheese')

In [6]:
df = pd.read_parquet('Halved-DF.parquet.gzip')

In [7]:
def data_query(df, ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue
        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})

    df_comb = pd.DataFrame(data)
    return df_comb

df_comb = data_query(df, combinations)
df_comb

,Combination,Score
0,"(onion, chicken)",[1.3248796627107513]
1,"(onion, cheese)",[1.4120038875676597]
2,"(onion, garlic)",[1.4235409132992736]
3,"(chicken, cheese)",[1.0177501859513405]
4,"(chicken, garlic)",[1.1800811339314616]
5,"(cheese, garlic)",[1.0944496012124034]
6,"[(onion, chicken), (onion, cheese), (chicken, ...","[1.3248796627107513, 1.4120038875676597, 1.017..."
7,"[(onion, chicken), (onion, garlic), (chicken, ...","[1.3248796627107513, 1.4235409132992736, 1.180..."
8,"[(onion, cheese), (onion, garlic), (cheese, ga...","[1.4120038875676597, 1.4235409132992736, 1.094..."
9,"[(chicken, cheese), (chicken, garlic), (cheese...","[1.0177501859513405, 1.1800811339314616, 1.094..."


In [8]:
def muse_comb(data_query_df): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     the function calculates the sum of the "Score" values and returns the three combinations with the largest sums
     OUTPUT: [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(data_query_df)):
        data_query_df["Sum"] = data_query_df["Score"].apply(calculate_sum)

    max_values = data_query_df.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists


In [9]:
lists = muse_comb(df_comb)
lists

[['onion', 'chicken', 'cheese', 'garlic'],
 ['onion', 'cheese', 'garlic'],
 ['onion', 'chicken', 'garlic']]

In [10]:
api_key = "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"
client = Groq(
api_key=api_key
)
recipe_list = []
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
        }
    ],
    model="llama3-8b-8192",
)

recipe = chat_completion.choices[0].message.content
recipe

2024-05-07 17:36:18.885 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'Here is a recipe that combines the given ingredients:\n\n**Cheesy Chicken Fritters**\n================================\n\n**Ingredients**\n\n* 1 medium onion, finely chopped\n* 1 lb boneless, skinless chicken breast or thighs, cut into small pieces\n* 1 cup shredded cheese (such as cheddar or mozzarella)\n* 3 cloves of garlic, minced\n\n**Directions**\n\n1. In a large bowl, combine the chopped onion, chicken, and garlic. Mix well until everything is fully incorporated.\n2. Divide the mixture into 4-6 portions, depending on the size you prefer for your fritters.\n3. Shape each portion into a ball and then flatten slightly into a disk shape.\n4. Place the shredded cheese in the center of each disk and fold the chicken mixture over the cheese to form a ball.\n5. Flatten each ball slightly and sandwich another small piece of cheese in the center.\n6. Heat about 1/2 inch of oil in a large skillet over medium-high heat. When hot, add the fritters and cook until golden brown on both sides, a

In [11]:
parts = recipe.split("**")
title = parts[1].strip()
ingredients = parts[4].strip()
directions = parts[6].strip()

recipe_dict = {}
recipe_dict['title'] = title
recipe_dict['ingredients'] = ingredients
recipe_dict['directions'] = directions
recipe_dict

{'title': 'Cheesy Chicken Fritters',
 'ingredients': '* 1 medium onion, finely chopped\n* 1 lb boneless, skinless chicken breast or thighs, cut into small pieces\n* 1 cup shredded cheese (such as cheddar or mozzarella)\n* 3 cloves of garlic, minced',
 'directions': '1. In a large bowl, combine the chopped onion, chicken, and garlic. Mix well until everything is fully incorporated.\n2. Divide the mixture into 4-6 portions, depending on the size you prefer for your fritters.\n3. Shape each portion into a ball and then flatten slightly into a disk shape.\n4. Place the shredded cheese in the center of each disk and fold the chicken mixture over the cheese to form a ball.\n5. Flatten each ball slightly and sandwich another small piece of cheese in the center.\n6. Heat about 1/2 inch of oil in a large skillet over medium-high heat. When hot, add the fritters and cook until golden brown on both sides, about 4-5 minutes per side.\n7. Serve hot and enjoy!'}

In [12]:
def recipe_generator(lists):
    api_key = "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"
    client = Groq(
    api_key=api_key
    )
    recipe_list = []
    for i in range(len(lists)):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Suggest one recipe with {lists[0]} only. The final format of the output should contain Title, Ingredients and Directions only",
                }
            ],
            model="llama3-8b-8192",
        )

        recipe = chat_completion.choices[0].message.content

        parts = recipe.split("**")
        title = parts[1].strip()
        ingredients = parts[4].strip()
        directions = parts[6].strip()

        recipe_dict = {}
        recipe_dict['title'] = title
        recipe_dict['ingredients'] = ingredients
        recipe_dict['directions'] = directions

        recipe_list.append(recipe_dict)

    return recipe_list


In [13]:
recipe_dicts = recipe_generator(lists)
recipe_dicts

2024-05-07 17:36:19.967 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-07 17:36:20.377 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-07 17:36:20.839 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


[{'title': 'Garlic Chicken Onion Frittata',
  'ingredients': '* 1 large onion, diced\n* 1 pound boneless, skinless chicken breast, cut into 1-inch pieces\n* 3 cloves of garlic, minced\n* 2 cups shredded cheese (cheddar or mozzarella work well)',
  'directions': '1. Preheat oven to 375°F.\n2. In a large skillet, cook the chicken and onion over medium-high heat until the chicken is cooked through and the onion is translucent.\n3. Add the garlic and cook for 1 minute.\n4. In a large bowl, whisk together 6 eggs and a pinch of salt.\n5. Add the cooked chicken and onion mixture to the bowl and stir to combine.\n6. In a greased 9-inch pie plate, add the egg mixture and top with the shredded cheese.\n7. Bake for 25-30 minutes or until the eggs are set and the cheese is melted and golden brown.'},
 {'title': 'Chicken and Cheese Fritters',
  'ingredients': '* 1 large onion, finely chopped\n* 1 lb boneless chicken breast, cut into small pieces\n* 1 cup shredded cheese\n* 3 cloves garlic, minced',

In [16]:
len(recipe_dicts)

3

In [52]:
def get_model():
    with open("model.pickle", "rb") as f:
        model = pickle.load(f)
    return model

model = get_model()

scores = []

for i in recipe_dicts:
    scores.append(model.predict_proba(i)[2][0])

print(scores)

[0.5356263108500366, 0.5356263108500366, 0.5356263108500366]


In [48]:
def final_recipes(recipe_dict, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    INPUT: Output of the recipe generator function
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    """
    final_recipes = {"Title": [], "Ingredients": [], "Instructions": []}
    threshold = 0.1

    for i in range(len(recipe_dict)):
        if scores[i] >= threshold:
            final_recipes["Title"].append([recipe['title'] for recipe in recipe_dicts][i])
            final_recipes["Ingredients"].append([recipe["ingredients"] for recipe in recipe_dicts][i])
            final_recipes["Instructions"].append([recipe["directions"] for recipe in recipe_dicts][i])
        else:
            n = 0
            tmp_recipe = {
                "Title":[recipe['title'] for recipe in recipe_dicts][i],
                "Ingredients":[recipe["ingredients"] for recipe in recipe_dicts][i],
                "Instructions":[recipe["directions"] for recipe in recipe_dicts][i]
                         }
            last_recipe = {
                "Title":[recipe['title'] for recipe in recipe_dicts][i],
                "Ingredients":[recipe["ingredients"] for recipe in recipe_dicts][i],
                "Instructions":[recipe["directions"] for recipe in recipe_dicts][i]
                         }
            while n < 3:
                new_recipe = recipe_generator(tmp_recipe["Ingredients"][0]) ###<=== insert actual recipe generator
                new_score = model.predict_proba(new_recipe["Instructions"][0]) ###<=== insert the actual scoring model function here
                if new_score >= threshold:
                    final_recipes["Title"].append(new_recipe["Title"])
                    final_recipes["Ingredients"].append(new_recipe["Ingredients"])
                    final_recipes["Instructions"].append(new_recipe["Instructions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe["Title"])
                final_recipes["Ingredients"].append(last_recipe["Ingredients"])
                final_recipes["Instructions"].append(last_recipe["Instructions"])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

In [53]:
main_dict = final_recipes(recipe_dicts, scores, model)
print(main_dict)

{'Title': ['Garlic Chicken Onion Frittata', 'Chicken and Cheese Fritters', 'Garlic Chicken and Onion Fritters'], 'Ingredients': ['* 1 large onion, diced\n* 1 pound boneless, skinless chicken breast, cut into 1-inch pieces\n* 3 cloves of garlic, minced\n* 2 cups shredded cheese (cheddar or mozzarella work well)', '* 1 large onion, finely chopped\n* 1 lb boneless chicken breast, cut into small pieces\n* 1 cup shredded cheese\n* 3 cloves garlic, minced', '* 1 large onion, finely chopped\n* 1 lb boneless, skinless chicken breast or thighs, cut into small pieces\n* 2 cloves of garlic, minced\n* 1 cup shredded cheese (any type)'], 'Instructions': ['1. Preheat oven to 375°F.\n2. In a large skillet, cook the chicken and onion over medium-high heat until the chicken is cooked through and the onion is translucent.\n3. Add the garlic and cook for 1 minute.\n4. In a large bowl, whisk together 6 eggs and a pinch of salt.\n5. Add the cooked chicken and onion mixture to the bowl and stir to combine.\

In [55]:
scores2 = [0.01, 0.5356263108500366, 0.5356263108500366]

In [56]:
main_dict = final_recipes(recipe_dicts, scores2, model)
print(main_dict)

2024-05-07 17:56:10.781 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 503 Service Unavailable"
2024-05-07 17:56:10.782 INFO    groq._base_client: Retrying request to /openai/v1/chat/completions in 5.000000 seconds
2024-05-07 17:56:16.370 INFO    httpx: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


IndexError: list index out of range